In [ ]:
cd to/your/path

### コードを github から clone する

In [ ]:
!git clone https://github.com/yuyaya-foifoi/ResearchAutoflow.git
%cd ResearchAutoflow

### 必要最低限のライブラリを install

In [ ]:
!pip install pymupdf PyPDF2 boto3 anthropic

### 実験に使う関数の import 

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('autores_openai')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('autores_anthropic')

from src.llm_interaction.experiment import generate_query, generate_initial_idea
from src.paper_analysis.arxiv import analyze_papers_to_df
from src.experiments.run_experiment import run_experiment

### 初期の coarse なアイデアの記述
 サンプルは https://huggingface.co/papers/2412.17739 の abst

In [ ]:
coarse_idea = """
Extending the context length of Language Models (LMs) by improving Rotary Position Embedding (RoPE) has become a trend. While existing works mainly address RoPE's limitations within attention mechanism, this paper provides an analysis across nearly all parts of LMs, uncovering their adverse effects on length generalization for RoPE-based attention. Using Discrete Signal Processing theory, we show that RoPE enables periodic attention by implicitly achieving Non-Uniform Discrete Fourier Transform. However, this periodicity is undermined by the spectral damage caused by: 1) linear layers and activation functions outside of attention; 2) insufficiently trained frequency components brought by time-domain truncation. Building on our observations, we propose Fourier Position Embedding (FoPE), which enhances attention's frequency-domain properties to improve both its periodic extension and length generalization. FoPE constructs Fourier Series and zero-outs the destructive frequency components, increasing model robustness against the spectrum damage. Experiments across various model scales show that, within varying context windows, FoPE can maintain a more stable perplexity and a more consistent accuracy in a needle-in-haystack task compared to RoPE and ALiBi. Several analyses and ablations bring further support to our method and theoretical modeling.
"""

### coarse なアイデアを発展させるための論文探索をするためのクエリを生成

In [ ]:
import numpy as np
import pandas as pd

queries = []

for i in np.arange(4):
    query_sets = generate_query(coarse_idea)
    queries.append(query_sets.idea)

### クエリをもとに survey する
中身としてはarxivを検索し、論文を取得し、その論文内に githubのlinkがあれば提案手法のpythonコードを取得する

In [ ]:
dfs = []
for query in queries:
    df = analyze_papers_to_df(query, num_papers=3)
    dfs.append(df)

initial_survey_df = pd.concat(dfs, ignore_index=True)

### survey の結果をもとに 初期のアイデアを発展させる

In [ ]:
idea = generate_initial_idea(
    idea=coarse_idea,
    survey_df=initial_survey_df
)
print(idea.idea)
print(idea.desc_of_idea)

### 実験を行う

In [ ]:
N_EXP = 10

# ---実験結果を s3 に保存しない場合 ---

run_experiment(
    idea=idea.idea,
    initial_survey_df=initial_survey_df,
    num_experiment=N_EXP,
    )

# ---実験結果を s3 に保存する場合 ---
"""
AWS_ACCESS_KEY = userdata.get('AWS_ACCESS_KEY')
AWS_SECRET_KEY = userdata.get('AWS_SECRET_KEY')
AWS_REGION = "us-east-1"
BUCKET_NAME = "auto-experiment-output"
S3_FOLDER = "experiments/"

run_experiment(
    idea=idea.idea,
    initial_survey_df=initial_survey_df,
    num_experiment=N_EXP,
    bucket=BUCKET_NAME,
    s3_folder=S3_FOLDER,
    access_key=AWS_ACCESS_KEY,
    secret_key=AWS_SECRET_KEY,
    region=AWS_REGION
    )
"""